In [28]:
from transformers import BartForConditionalGeneration, BartTokenizer
import pandas as pd

In [29]:
# Load the model and tokenizer
model = BartForConditionalGeneration.from_pretrained(
    'facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained(
    'facebook/bart-large-cnn')

Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-large-cnn and are newly initialized: ['model.shared.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
def summarize(text, maxSummarylength=500):
    # Encode the text and summarize
    inputs = tokenizer.encode("summarize: " +
                              text,
                              return_tensors="pt",
                              max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=maxSummarylength,
                                 min_length=int(maxSummarylength/5),
                                 length_penalty=10.0,
                                 num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [31]:
def split_text_into_pieces(text,
                           max_tokens=900,
                           overlapPercent=10):
    # Tokenize the text
    tokens = tokenizer.tokenize(text)

    # Calculate the overlap in tokens
    overlap_tokens = int(max_tokens * overlapPercent / 100)

    # Split the tokens into chunks of size
    # max_tokens with overlap
    pieces = [tokens[i:i + max_tokens]
              for i in range(0, len(tokens),
                             max_tokens - overlap_tokens)]

    # Convert the token pieces back into text
    text_pieces = [tokenizer.decode(
        tokenizer.convert_tokens_to_ids(piece),
        skip_special_tokens=True) for piece in pieces]

    return text_pieces

In [32]:
def recursive_summarize(text, max_length=200, recursionLevel=0):
    recursionLevel=recursionLevel+1
    print("######### Recursion level: ",
          recursionLevel,"\n\n######### ")
    tokens = tokenizer.tokenize(text)
    expectedCountOfChunks = len(tokens)/max_length
    max_length=int(len(tokens)/expectedCountOfChunks)+2

    # Break the text into pieces of max_length
    pieces = split_text_into_pieces(text, max_tokens=max_length)

    print("Number of pieces: ", len(pieces))
    # Summarize each piece
    summaries=[]
    k=0
    for k in range(0, len(pieces)):
        piece=pieces[k]
        print("****************************************************")
        print("Piece:",(k+1)," out of ", len(pieces), "pieces")
        print(piece, "\n")
        summary =summarize(piece, maxSummarylength=max_length/3*2)
        print("SUMNMARY: ", summary)
        summaries.append(summary)
        print("****************************************************")

    concatenated_summary = ' '.join(summaries)

    tokens = tokenizer.tokenize(concatenated_summary)

    if len(tokens) > max_length:
        # If the concatenated_summary is too long, repeat the process
        print("############# GOING RECURSIVE ##############")
        return recursive_summarize(concatenated_summary,
                                   max_length=max_length,
                                   recursionLevel=recursionLevel)
    else:
      # Concatenate the summaries and summarize again
        final_summary=concatenated_summary
        if len(pieces)>1:
            final_summary = summarize(concatenated_summary,
                                  maxSummarylength=max_length)
        return final_summary

In [33]:
train_data = pd.read_csv('Training_data.csv')
validate_data = pd.read_csv('Validation_data.csv')
test_data = pd.read_csv('Testing_data.csv')

train_data = train_data[['summary_id','chapter','chapter_length','summary_name','summary_text','summary_analysis','summary_length','analysis_length']]
validate_data = validate_data[['summary_id','chapter','chapter_length','summary_name','summary_text','summary_analysis','summary_length','analysis_length']]
test_data = test_data[['summary_id','chapter','chapter_length','summary_name','summary_text','summary_analysis','summary_length','analysis_length']]

In [34]:
text = train_data['chapter'][0]

In [35]:
final_summary = recursive_summarize(text)
print("\n%%%%%%%%%%%%%%%%%%%%%\n")
print("Final summary:", final_summary)

######### Recursion level:  1 

######### 
Number of pieces:  44
****************************************************
Piece: 1  out of  44 pieces
mine ear is open and my heart prepared the worst is worldly loss thou canst unfold say is my kingdom lost shakespeare it was a feature peculiar to the colonial wars of north america that the toils and dangers of the wilderness were to be encountered before the adverse hosts could meet a wide and apparently an impervious boundary of forests severed the possessions of the hostile provinces of france and england the hardy colonist and the trained european who fought at his side frequently expended months in struggling against the rapids of the streams or in effecting the rugged passes of the mountains in quest of an opportunity to exhibit their courage in a more martial conflict but emulating the patience and self-denial of the practised native warriors they learned to overcome every difficulty and it would seem that in time there was no recess 